In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler


realCIR = np.load('real.npy')
imagCIR = np.load('imag.npy')
magCIR = np.load('dataset.npy')
GT = np.load('label.npy')

print(realCIR.shape, imagCIR.shape, magCIR.shape, GT.shape)

tan = np.arctan2(imagCIR, realCIR)
diff = tan[:,:63] - tan[:, 1:]
diffrence = np.zeros((magCIR.shape[0],magCIR.shape[1]))
diffrence[:,1:] = diff

array = np.stack([magCIR,diffrence],axis=-1)
# array = np.stack([magCIR,tan],axis=-1)

X_train, X_test, y_train, y_test = train_test_split(array, GT, test_size=0.2, shuffle=True, stratify=GT)

(18000, 64) (18000, 64) (18000, 64) (18000,)


In [2]:
class ResNet18(tf.keras.Model):

    def __init__(self):
        super().__init__()
        self.convA1 = tf.keras.layers.Conv1D(64,7,2,activation=tf.nn.relu,input_shape=(64,2),padding='same')
        self.normA1 = tf.keras.layers.BatchNormalization()
        self.poolA1 = tf.keras.layers.MaxPooling1D(2,padding='same')
        
        self.convB1 = tf.keras.layers.Conv1D(64,3,1,padding='same')
        self.convB2 = tf.keras.layers.Conv1D(64,3,1,padding='same')
        self.convB3 = tf.keras.layers.Conv1D(64,3,1,padding='same')
        self.convB4 = tf.keras.layers.Conv1D(64,3,1,padding='same')
        self.normB1 = tf.keras.layers.BatchNormalization()
        self.normB2 = tf.keras.layers.BatchNormalization()
        self.poolB1 = tf.keras.layers.MaxPooling1D(2,padding='same')
        
        self.convC1 = tf.keras.layers.Conv1D(128,3,1,padding='same')
        self.convC2 = tf.keras.layers.Conv1D(128,3,1,padding='same')
        self.convC3 = tf.keras.layers.Conv1D(128,3,1,padding='same')
        self.convC4 = tf.keras.layers.Conv1D(128,3,1,padding='same')
        self.normC1 = tf.keras.layers.BatchNormalization()
        self.normC2 = tf.keras.layers.BatchNormalization()
        self.poolC1 = tf.keras.layers.MaxPooling1D(2,padding='same')
        self.shortcutC = tf.keras.layers.Conv1D(128,1,1,padding='same')
        
        self.convD1 = tf.keras.layers.Conv1D(256,3,1,padding='same')
        self.convD2 = tf.keras.layers.Conv1D(256,3,1,padding='same')
        self.convD3 = tf.keras.layers.Conv1D(256,3,1,padding='same')
        self.convD4 = tf.keras.layers.Conv1D(256,3,1,padding='same')
        self.normD1 = tf.keras.layers.BatchNormalization()
        self.normD2 = tf.keras.layers.BatchNormalization()
        self.poolD1 = tf.keras.layers.MaxPooling1D(2,padding='same')
        self.shortcutD = tf.keras.layers.Conv1D(256,1,1,padding='same')
        
        self.convE1 = tf.keras.layers.Conv1D(512,3,1,padding='same')
        self.convE2 = tf.keras.layers.Conv1D(512,3,1,padding='same')
        self.convE3 = tf.keras.layers.Conv1D(512,3,1,padding='same')
        self.convE4 = tf.keras.layers.Conv1D(512,3,1,padding='same')
        self.normE1 = tf.keras.layers.BatchNormalization()
        self.normE2 = tf.keras.layers.BatchNormalization()
        self.poolE1 = tf.keras.layers.MaxPooling1D(2,padding='same')
        self.shortcutE = tf.keras.layers.Conv1D(512,1,1,padding='same')
        
        self.normF = tf.keras.layers.AveragePooling1D()
        self.flatten = tf.keras.layers.Flatten()
        self.denseF1 = tf.keras.layers.Dense(1000,activation=tf.nn.relu)
        self.denseF2 = tf.keras.layers.Dense(6,activation=tf.nn.softmax)
#     def conv_block(input_tensor):
#         x = tf.keras.layers.Conv1D(64,3,1,padding='same')(input_tensor)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.nn.relu(x)
#         x = tf.keras.layers.Conv1D(64,3,1,padding='same')(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         sc = tf.keras.layers.Conv1D(64,3,1,padding='same')(input_tensor)
#         sc = tf.keras.layers.BatchNormalization()(sc)
#         return x+sc
    
    def call(self, inputs):
        # blockA
        x = self.convA1(inputs)
        x = self.normA1(x)
        x = self.poolA1(x)
        # blockB
        con_b = self.convB1(x)
        con_b = self.normB1(con_b)
        con_b = tf.keras.activations.relu(con_b)
        con_b = self.convB2(con_b)
        x += con_b
        x = tf.keras.activations.relu(x)
        con_b = self.convB3(x)
        con_b = self.normB2(con_b)
        con_b = tf.keras.activations.relu(con_b)
        con_b = self.convB4(con_b)
        x += con_b
        x = tf.keras.activations.relu(x)
        x = self.poolB1(x)
        # blockC
        con_c = self.convC1(x)
        con_c = self.normC1(con_c)
        con_c = tf.keras.activations.relu(con_c)
        con_c = self.convC2(con_c)
        x = self.shortcutC(x)
        x += con_c
        x = tf.keras.activations.relu(x)
        con_c = self.convC3(x)
        con_c = self.normC2(con_c)
        con_c = tf.keras.activations.relu(con_c)
        con_c = self.convC4(con_c)
        x += con_c
        x = tf.keras.activations.relu(x)
        x = self.poolC1(x)
        # blockD
        con_d = self.convD1(x)
        con_d = self.normD1(con_d)
        con_d = tf.keras.activations.relu(con_d)
        con_d = self.convD2(con_d)
        x = self.shortcutD(x)
        x += con_d
        x = tf.keras.activations.relu(x)
        con_d = self.convD3(x)
        con_d = self.normD2(con_d)
        con_d = tf.keras.activations.relu(con_d)
        con_d = self.convD4(con_d)
        x += con_d
        x = tf.keras.activations.relu(x)
        x = self.poolD1(x)
        # blockE
        con_e = self.convE1(x)
        con_e = self.normE1(con_e)
        con_e = tf.keras.activations.relu(con_e)
        con_e = self.convE2(con_e)
        x = self.shortcutE(x)
        x += con_e
        x = tf.keras.activations.relu(x)
        con_e = self.convE3(x)
        con_e = self.normE1(con_e)
        con_e = tf.keras.activations.relu(con_e)
        con_e = self.convE4(con_e)
        x += con_e
        x = tf.keras.activations.relu(x)
        # blockF
        x = self.normF(x)
        x = self.flatten(x)
        x = self.denseF1(x)
        x = self.denseF2(x)
        return x

In [3]:
model = ResNet18()

2022-11-16 19:59:25.878980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def step_decay(epoch):
    start = 0.0001
    drop = 0.1
    epochs_drop = 5
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

In [5]:
lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

In [6]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [8]:
model.build(input_shape = (None,64,2))

In [9]:
model.summary()

Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  960       
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  multiple                 0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           multiple                  12352     
                                                                 
 conv1d_2 (Conv1D)           multiple                  12352     
                                                                 
 conv1d_3 (Conv1D)           multiple                  12

In [10]:
model.fit(X_train, y_train, epochs=10,
            validation_data=(X_test, y_test),
             callbacks=[lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/10
450/450 [==============================] - 25s 54ms/step - loss: 0.9824 - accuracy: 0.6008 - val_loss: 0.8659 - val_accuracy: 0.6600 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/10
450/450 [==============================] - 24s 53ms/step - loss: 0.6282 - accuracy: 0.7547 - val_loss: 0.7451 - val_accuracy: 0.7094 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/10
450/450 [==============================] - 25s 55ms/step - loss: 0.4837 - accuracy: 0.8099 - val_loss: 0.6658 - val_accuracy: 0.7511 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/10
450/450 [==============================] - 24s 52ms/step - loss: 0.3890 - accuracy: 0.8536 - val_loss: 0.7353 - val_accuracy: 0.7408 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/10
450/450 [===========

In [11]:
model.evaluate(X_test, y_test)

113/113 [==============================] - 1s 9ms/step - loss: 0.7599 - accuracy: 0.7886


[0.7598958611488342, 0.788611114025116]

In [ ]:
def identity_block(input_tensor,filters,kernel_size,strides):
    x = tf.keras.layers.Conv1D(filters,kernel_size,strides,padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)
    x = tf.keras.layers.Conv1D(filters,kernel_size,strides,padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    return x + input_tensor

In [ ]:
    def conv_block(input_tensor):
        x = tf.keras.layers.Conv1D(64,3,1,padding='same')(input_tensor)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.nn.relu(x)
        x = tf.keras.layers.Conv1D(64,3,1,padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        sc = tf.keras.layers.Conv1D(64,3,1,padding='same')(input_tensor)
        sc = tf.keras.layers.BatchNormalization()(sc)
        return x+sc

In [ ]:
def conv_block(input_tensor,filters,kernel_size,strides):
    x = tf.keras.layers.Conv1D(filters,kernel_size,strides,padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)
    x = tf.keras.layers.Conv1D(filters,kernel_size,strides,padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    sc = tf.keras.layers.Conv1D(filters,kernel_size,strides,padding='same')(input_tensor)
    sc = tf.keras.layers.BatchNormalization()(sc)
    return x+sc

In [ ]:
"""ResNet50 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](
    https://arxiv.org/abs/1512.03385) (CVPR 2016 Best Paper Award)

Adapted from code contributed by BigMoyan.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

from . import get_submodules_from_kwargs
from . import imagenet_utils
from .imagenet_utils import decode_predictions
from .imagenet_utils import _obtain_input_shape

preprocess_input = imagenet_utils.preprocess_input

WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/'
                'releases/download/v0.2/'
                'resnet50_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

backend = None
layers = None
models = None
keras_utils = None


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.

    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names

    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.

    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.

    # Returns
        Output tensor for the block.

    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000,
             **kwargs):
    """Instantiates the ResNet50 architecture.

    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.

    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.

    # Returns
        A Keras model instance.

    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      data_format=backend.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = layers.Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)
        else:
            warnings.warn('The output shape of `ResNet50(include_top=False)` '
                          'has been changed since Keras 2.2.0.')

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = models.Model(inputs, x, name='resnet50')

    # Load weights.
    if weights == 'imagenet':
        if include_top:
            weights_path = keras_utils.get_file(
                'resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,
                cache_subdir='models',
                md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = keras_utils.get_file(
                'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,
                cache_subdir='models',
                md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if backend.backend() == 'theano':
            keras_utils.convert_all_kernels_in_model(model)
    elif weights is not None:
        model.load_weights(weights)

    return model